In [23]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras import layers, Model
from osgeo import gdal
import os
import json

File opened successfully!


In [14]:
### Load Images

def load_image(file_path):
    dataset = gdal.Open("/~/Documents/JADS/DeepLearning/SEN12FLOOD/0307")
    if dataset is None:
        raise FileNotFoundError("Could not open file:", file_path)
    array = dataset.ReadAsArray()
    return array

In [24]:
### Load JSON Files (Flooding Boolean)

with open('/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/S1list.json', 'r') as f:
    flooding_events_s1 = json.load(f)
    
with open('/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/S2list.json', 'r') as satelites:
    flooding_events_s2 = json.load(satelites)


In [25]:
### Extract flooding boolean information 

json_folder_path = '/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD'

def load_json(file_path):
    with open(file_path, 'r') as file: 
        return json.load(file)



sequence_id = "0076"

flooding_info = {}

s1_json_file_path = '/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/S1list.json'
s2_json_file_path = '/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/S2list.json'

s1_data = load_json(s1_json_file_path)
s2_data = load_json(s2_json_file_path)

for key, value in s1_data[sequence_id].items():
    if key.isdigit():
        flooding_info[value["date"]] = value["FLOODING"]
        
for key, value in s2_data[sequence_id].items():
    if key.isdigit():
        flooding_info[value["date"]] = value["FLOODING"]
        
        # Print flooding information
for date, flooding in flooding_info.items():
    print("Date:", date, "Flood Event Present:", flooding)

Date: 2019-02-07 Flood Event Present: False
Date: 2019-02-03 Flood Event Present: False
Date: 2019-02-02 Flood Event Present: False
Date: 2019-01-26 Flood Event Present: False
Date: 2019-01-22 Flood Event Present: False
Date: 2019-01-16 Flood Event Present: False
Date: 2019-01-14 Flood Event Present: False
Date: 2019-01-10 Flood Event Present: False
Date: 2019-01-09 Flood Event Present: False
Date: 2019-01-04 Flood Event Present: False
Date: 2019-01-02 Flood Event Present: False
Date: 2018-12-29 Flood Event Present: False
Date: 2018-12-28 Flood Event Present: False
Date: 2018-12-26 Flood Event Present: False
Date: 2018-12-31 Flood Event Present: False
Date: 2019-01-05 Flood Event Present: False
Date: 2019-01-30 Flood Event Present: False
Date: 2019-02-04 Flood Event Present: False
Date: 2019-02-14 Flood Event Present: False


In [20]:
from osgeo import gdal
import os

# Function to load image
def load_image(file_path):
    return gdal.Open(file_path)

# Load satellite images
folder_path = '/home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076'

multispectral_images = []
sar_images = []

for file_name in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file_name)
    
    # Print file path for debugging
    print("Processing file:", file_path)
    
    if os.path.isfile(file_path):
        try:
            # Open the file
            dataset = load_image(file_path)
            if dataset is None:
                print("Could not open file:", file_path)
            else:
                print("File opened successfully!")
                # Check if the file is multispectral or SAR
                if file_name.startswith('S1'):
                    multispectral_images.append(dataset)
                elif file_name.startswith('S2'):
                    sar_images.append(dataset)
                else:
                    print("Unknown image type for file:", file_path)
        except Exception as e:
            print("Error loading file:", file_path)
            print("Error message:", str(e))

multispectral_images = np.array(multispectral_images)
sar_images = np.array(sar_images)

# Display the shapes of the arrays
print("Multispectral Images Shape:", multispectral_images.shape)
print("SAR Images Shape:", sar_images.shape)


Processing file: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S1B_IW_GRDH_1SDV_20190122T032731_20190122T032756_014601_01B34A_9A7F_corrected_VV.tif
File opened successfully!
Processing file: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S2_2018-12-26_B12.tif
File opened successfully!
Processing file: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S1A_IW_GRDH_1SDV_20190207T160454_20190207T160519_025825_02DF8C_59F6_corrected_VV.tif
File opened successfully!
Processing file: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S2_2018-12-26_B07.tif
File opened successfully!
Processing file: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S1A_IW_GRDH_1SDV_20190219T160454_20190219T160519_026000_02E5CA_4AA3_corrected_VH.tif
File opened successfully!
Processing file: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S2_2019-02-14_B12.tif
File opened successfully!
Processing file: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S1B_IW_GRDH_

Warning 1: /home/pedro/Documents/JADS/DeepLearning/SEN12FLOOD/0076/S1A_IW_GRDH_1SDV_20190114T160455_20190114T160520_025475_02D2DC_7BCF_corrected_VH.tif: TIFFReadDirectory:Bogus "StripByteCounts" field, ignoring and calculating from imagelength


In [ ]:
def create_model(input_shape_multispectral, input_shape_sar):
    #MultispectralBranch
    input_multispectral = layers.Input(shape=input_shape_multispectral)
    conv1_multispectral = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(input_multispectral)
    #SAR branch
    input_sar = layers.Input(shape=input_shape_sar)
    conv1_sar = layers.Conv2D(32, kernel_size=(3, 3), activation='relu')(input_sar)
    #Merge Branch
    merged = layers.concatenate([conv1_multispectral, conv1_sar])
    #Connected layers
    flat = layers.Flatten()(merged)
    dense1 = layers.Dense(128, activation='relu')(flat)
    output = layers.Dense(1, activation='sigmoid')(dense1)
    
    model = Model(inputs=[input_multispectral, input_sar], outputs=output)
    return model 

model = create_model(input_shape_multispectral=(12, 64, 64), input_shape_sar=(2, 64, 64))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit([X_train_multispectral, X_train_sar], y_train,
                     validation_data=([X_val_multispectral, X_val_sar], y_val),
                     epochs=10, batch_size=32)

test_loss, test_accuracy = model.evaluate([X_test_multispectral, X_test_sar], y_test)

print("Test Accuracy:", test_accuracy)